# B: 二重ハッシュ（**配列1始まり**を Python で再現）

In [ ]:

# --- Ensure ipywidgets is available (JupyterLite/regular Jupyter) ---
try:
    import ipywidgets as widgets
except ModuleNotFoundError:
    try:
        import piplite
        await piplite.install(['ipywidgets'])
    except Exception as e:
        print("piplite install failed or not JupyterLite:", e)
    import ipywidgets as widgets


In [ ]:

# === IPA擬似言語に寄せるためのユーティリティ ===
import math
from typing import List, Iterable

def range_inclusive(a: int, b: int, step: int = 1) -> Iterable[int]:
    """for (i を a から b まで step ずつ) の完全対応"""
    if step == 0: raise ValueError("step must be non-zero")
    if (a <= b and step > 0):
        return range(a, b+1, step)
    if (a >= b and step < 0):
        return range(a, b-1, step)
    return range(0)  # empty

def isqrt(n: int) -> int:
    """⌊√n⌋"""
    return math.isqrt(n)

class OneBasedList:
    """配列[1..N] を Python で表現（1始まり）"""
    def __init__(self, length: int, fill=None):
        self._data = [None] + [fill]*length  # index 1..length
        self.length = length
    def __getitem__(self, i: int):
        return self._data[i]
    def __setitem__(self, i: int, v):
        self._data[i] = v
    def to_list(self):
        return self._data[1:]


### 1) 擬似言語

In [ ]:
[H1] T を長さ5の配列(-1で初期化) とする（添字は1始まり）
[H2] h1(x) = (x mod 5) + 1, h2(x) = ((x+3) mod 5) + 1
[H3] add(v):
[H4]   if T[h1(v)] が空(-1)なら T[h1(v)] ← v
[H5]   else if T[h2(v)] が空なら T[h2(v)] ← v
[H6]   else 失敗


### 2) Python（行番号対応）

In [ ]:
# [H1] 配列（1始まり）
T = OneBasedList(5, fill=-1)
# [H2] ハッシュ関数（1始まりになるよう +1）
def h1(x): return (x % 5) + 1
def h2(x): return ((x + 3) % 5) + 1

# [H3] add(v)
def add(v):
    # [H4] まず h1
    p1 = h1(v)
    if T[p1] == -1:
        T[p1] = v
        return ("OK", p1)
    # [H5] 次に h2
    p2 = h2(v)
    if T[p2] == -1:
        T[p2] = v
        return ("OK(代替)", p2)
    # [H6] 両方埋まり
    return ("失敗", None)

for v in (3,18,11):
    add(v)
T.to_list()


### 3) 値を変えてみる

In [ ]:

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

vals = widgets.SelectMultiple(options=[3,11,18,21,26,31], value=(3,18,11), description="挿入")
out = widgets.Output()

def reset():
    global T
    T = OneBasedList(5, fill=-1)

def run(*_):
    with out:
        clear_output()
        reset()
        rows = []
        for v in vals.value:
            status, pos = add(v)
            rows.append((v, status, pos))
        html = "<table><tr><th>v</th><th>結果</th><th>格納位置</th></tr>" + \
               "".join(f"<tr><td>{a}</td><td>{b}</td><td>{c}</td></tr>" for a,b,c in rows) + "</table>"
        display(HTML(html))
        display(HTML("<b>T (1始まり)</b>: " + str(T.to_list())))

display(vals, out); run()
vals.observe(run, names="value")
